<td>
   <a target="_blank" href="https://beewant.com" ><img src="https://media.licdn.com/dms/image/C4E0BAQFKjwZohF7zRA/company-logo_200_200/0/1655303818792?e=2147483647&v=beta&t=fmLrigj1HhqskB4jNk6aDb6XASTUx1SnRvMXWBPtuCY"></a>
</td>
<p><B>Welcome to Beewant Platform Tutorial</B></p>

<td>
    <a href="https://www.linkedin.com/in/beewant/" target="_blank">
        <img src="https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white" alt="LinkedIn">
    </a>
</td>

<td>
    <a href="https://beewant.slack.com/" target="_blank">
        <img src="https://img.shields.io/badge/Slack-4A154B?logo=slack&logoColor=white" alt="Slack">
    </a>
</td>

<p>In this tutorial, we will guide you through the process of deleting tasks from both the S3 storage and the BeeWant platform, focusing on a specific set of values within the tags associated with tasks in a designated project. Learn how to efficiently manage and clean up your project data by leveraging the capabilities of BeeWant's platform and S3 storage. By following this tutorial, you'll gain insights into targeted task deletion based on customized tags, optimizing your project's task management process.</p>

In [ ]:
import requests
from tqdm import tqdm
import boto3
from urllib.parse import urlparse

# API Key and Client

#### Provide a valid api key below in order to properly connect to the Beewant Client.

In [ ]:
# To get your API key go to: Workspace settings -> Account -> Access Token
beewant_api_key = ""

# A project ID is a unique identifier assigned to each project in Beewant
current_project = ""

## Set aws credentials

In [ ]:
# Replace 'abc' with your access key and 'def' with your secret key
access_key = ''
secret_key = ''
region_name = ''

# Initialize an S3 client
s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name = region_name)

# Specify the bucket name
bucket_name = ''

## Import tasks from platform

#### set platform credentials

In [ ]:
url = f"https://beewant.com/api/projects/{current_project}/tasks/?page=1&page_size=-1"

headers = {'Authorization': f'Token {beewant_api_key}',}

tasks = requests.request("GET", url, headers=headers)
tasks=tasks.json()

In [ ]:
# filtering tasks with 'delete' tag
filtered_tasks = [task for task in tasks if any(
    'delete' in tag for tag in task['tags']
)]

print("Number of filtered tasks: ",len(filtered_tasks))

In [ ]:

# creating of a dict of tasks with task_id as key and image url as value
task_dict = {task['id']: task['data']['image'] for task in filtered_tasks}

In [ ]:
print("dictionnary of tasks ids and their path in s3:",task_dict)

## Deleting from aws storage

In [ ]:
def delete_s3_files(bucket_name, files_to_delete, access_key, secret_key,region_name):
    # Create an S3 client
    s3 = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
    
    # Iterate over the list of files to delete
    for file_path in tqdm(files_to_delete, desc="Deleting files", unit="file"):
        # Parse the S3 URL to get the bucket name and file key
        parsed_url = urlparse(file_path)
        bucket_name = parsed_url.netloc
        file_key = parsed_url.path[1:]  # Remove the leading '/' from the path

        try:
            # Delete the file from the S3 bucket
            s3.delete_object(Bucket=bucket_name, Key=file_key)
            #print(f"Deleted file: {file_key}")
        except Exception as e:
            print(f"Error deleting file {file_key}: {str(e)}")

In [ ]:
# delete the files from s3
delete_s3_files(bucket_name, task_dict.values(),access_key,secret_key,region_name)

In [ ]:
# iterate over the tasks in the dict and replace the format of image (jpeg,jpg,webp,png) by json
for task_id, image_url in tqdm(task_dict.items(), desc="Replacing image format by json", unit="task"):
    # you can add other formats if needed, in this case we are working with images
    task_dict[task_id] = image_url.replace(".jpeg", ".json").replace(".jpg", ".json").replace(".png", ".json").replace(".webp", ".json")

# delete the json files from s3
delete_s3_files(bucket_name, task_dict.values(),access_key,secret_key,region_name)

# Deleting tasks from the platform

In [ ]:
# delete these tasks from platform
for task_id in tqdm(task_dict.keys(), desc="Deleting tasks", unit="task"):
    url = f"https://beewant.com/api/tasks/{task_id}/"
    response = requests.request("DELETE", url, headers=headers)
    print(f"Deleted task: {task_id}")